In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df = pd.read_excel('./cleaned/CleanedSubmissions.xlsx')
df = (
pd.DataFrame({
    'company_id': df['CompanyID'].to_list() * 5,
    'company': df['Company'].to_list() * 5,
    'volume': df['2019 Volume'].to_list() + df['2020 Volume'].to_list() + df['2021 Volume'].to_list() + df['2022 Volume'].to_list() + df['2023 Volume'].to_list(),
    'sides': df['2019 Sides'].to_list() + df['2020 Sides'].to_list() + df['2021 Sides'].to_list() + df['2022 Sides'].to_list() + df['2023 Sides'].to_list(),
    'referrer': df['Referral Network'].to_list() * 5,
    'year': [2019] * len(df['2019 Volume']) + [2020] * len(df['2020 Volume']) + [2021] * len(df['2021 Volume']) + [2022] * len(df['2022 Volume']) + [2023] * len(df['2023 Volume']) ,
    # 'city': df['City'].to_list() * 5,
    'state': df['State'].to_list() * 5, 
    # 'country': df['Country Code'].to_list() * 5, 
    'logo': df['Logo'].to_list() * 5
})
)
df['pct_volume'] = df['volume'] / df.groupby('year')['volume'].transform('sum') * 100
df['pct_sides'] = df['sides'] / df.groupby('year')['sides'].transform('sum') * 100

In [3]:
volume_race = df.loc[df.company_id.isin(df.loc[df.volume.isin(df.groupby('year').volume.nlargest(n=10)), 'company_id'].unique())].pivot(
                    index=['company_id', 'company', 'logo'], columns='year', values=['volume', 'pct_volume']
                )
volume_race.columns = volume_race.columns.droplevel()
volume_race.columns.name = ''
volume_race.to_excel('./flourish/LineChartRace_Top10.xlsx')

In [4]:
sides_race = df.loc[df.company_id.isin(df.loc[df.sides.isin(df.groupby('year').sides.nlargest(n=10)), 'company_id'].unique())].pivot(
                    index=['company_id', 'company', 'logo'], columns='year', values=['sides', 'pct_sides']
                )
sides_race.columns = sides_race.columns.droplevel()
sides_race.columns.name = ''
sides_race.to_excel('./flourish/LineChartRace_Top10_Sides.xlsx')

In [5]:
share_referred = df.copy()[['year','referrer', 'volume', 'sides']]
share_referred['referrer'] = share_referred.referrer.apply(lambda x: 'Major Referrers' if x != 'Independent/Other' else x)
share_referred = share_referred.groupby(['year', 'referrer']).sum().reset_index()
share_referred['volume'] = share_referred['volume'] / 1000000000
share_referred['sides'] = share_referred['sides'] / 1000
share_referred = share_referred.pivot(index='year', columns='referrer', values=['volume', 'sides'])
share_referred.columns = share_referred.columns.droplevel()
share_referred.columns.name = ''
share_referred = pd.concat([share_referred.iloc[:,:2], share_referred.iloc[:,2:]])
share_referred['group'] = ['Volume (billions of $)'] * 5 + ['Sides (thousands)'] * 5
share_referred.to_excel('./flourish/ShareReferred.xlsx')

In [6]:
hsa = df.copy().loc[(df.year==2023)&(df.company=='HomeServices of America'), ['company_id', 'company', 'referrer', 'volume', 'sides']]
hsa['referrer'] = [x.split(',') for x in hsa.referrer]
hsa = hsa.explode('referrer').reset_index(drop=True)
hsa.loc[hsa.referrer=='Berkshire Hathaway HomeServices', 'volume'] = 55598689740 
hsa.loc[hsa.referrer=='Berkshire Hathaway HomeServices', 'sides'] = 90533 
hsa.loc[hsa.referrer=='LeadingRE', 'volume'] = 77495317918 
hsa.loc[hsa.referrer=='LeadingRE', 'sides'] = 143645 
# add Indy
hsa = pd.concat([hsa, pd.DataFrame({'company_id': [hsa.company_id.min()],
                              'company': [hsa.company.min()],
                              'referrer': ['Other'],
                              'volume': [712707712 ],
                              'sides': [1683 ]})], ignore_index=True)

tomlinson = df.copy().loc[(df.year==2023)&(df.company.str.contains('Tomlinson')), ['company_id', 'company', 'referrer', 'volume', 'sides']]
tomlinson['referrer'] = [x.split(',') for x in tomlinson.referrer]
tomlinson = tomlinson.explode('referrer').reset_index(drop=True)
tomlinson.loc[tomlinson.referrer=='CENTURY 21', 'volume'] = 568810771 
tomlinson.loc[tomlinson.referrer=='CENTURY 21', 'sides'] = 833 
tomlinson.loc[tomlinson.referrer=='Coldwell Banker', 'volume'] = 1478825889 
tomlinson.loc[tomlinson.referrer=='Coldwell Banker', 'sides'] = 3302 
tomlinson.loc[tomlinson.referrer=='Sotheby\'s International Realty', 'volume'] = 959482647 
tomlinson.loc[tomlinson.referrer=='Sotheby\'s International Realty', 'sides'] = 1007 

hrc = df.copy().loc[(df.year==2023)&(df.company.str.contains('HRC')), ['company_id', 'company', 'referrer', 'volume', 'sides']]
hrc['referrer'] = [x.split(',') for x in hrc.referrer]
hrc = hrc.explode('referrer').reset_index(drop=True)
hrc.loc[hrc.referrer=='CENTURY 21', 'volume'] = 517245607 
hrc.loc[hrc.referrer=='CENTURY 21', 'sides'] = 1575 
hrc.loc[hrc.referrer=='Coldwell Banker', 'volume'] = 333213199 
hrc.loc[hrc.referrer=='Coldwell Banker', 'sides'] = 1555 

anywhere = df.copy().loc[(df.year==2023)&(df.company.str.contains('Anywhere Advisors')), ['company_id', 'company', 'referrer', 'volume', 'sides']]
anywhere['referrer'] = [x.split(',') for x in anywhere.referrer]
anywhere = anywhere.explode('referrer').reset_index(drop=True)
anywhere.loc[anywhere.referrer=='Coldwell Banker', 'volume'] = 143003776000 
anywhere.loc[anywhere.referrer=='Coldwell Banker', 'sides'] = 234969 
anywhere.loc[anywhere.referrer=='Corcoran', 'volume'] = 15306481000 
anywhere.loc[anywhere.referrer=='Corcoran', 'sides'] = 7375 
anywhere.loc[anywhere.referrer=='Sotheby\'s International Realty', 'volume'] = 17919208000 
anywhere.loc[anywhere.referrer=='Sotheby\'s International Realty', 'sides'] = 9679 

multis = pd.concat([hsa, tomlinson, hrc, anywhere], ignore_index=True)

top_referrers = pd.concat([multis, 
                    df.loc[(df.year==2023)&(df.referrer!='Independent/Other')&(~df.company.isin(multis.company.unique())), 
                            ['company_id', 'referrer', 'volume', 'sides']]]).groupby('referrer').agg({
                                'company_id':'nunique',
                                'volume':'sum',
                                'sides':'sum'
                            }).sort_values(by='volume', ascending=False)
top_referrers['volume'] = top_referrers['volume'].astype('int64')
top_referrers['sides'] = top_referrers['sides'].astype('int64')

top_referrers = top_referrers.iloc[:-1,:]

top_referrers['volume'] = round(top_referrers.volume / 1000000000,2)
top_referrers['sides'] = round(top_referrers.sides / 1000,2)

top_referrers['volume_per_company'] = top_referrers.volume / top_referrers.company_id 
top_referrers['sides_per_company'] = top_referrers.sides / top_referrers.company_id

top_referrers.to_excel('./flourish/TopReferrers2023.xlsx')

In [7]:
# not independent/other and not null
just_referrers = df.dropna(how='any')
just_referrers = just_referrers.loc[just_referrers.referrer!='Independent/Other', ['year', 'state', 'referrer']]
# clean state column
just_referrers.loc[just_referrers.state=='Arizona', 'state'] = 'AZ'
# address companies with multiple affiliations
just_referrers['referrer'] = just_referrers.referrer.apply(lambda x: x.split(','))
just_referrers = just_referrers.explode('referrer')

just_referrers = pd.DataFrame(just_referrers.value_counts()).sort_values(by=['year', 'state', 'count'], ascending=[True, True, False]).reset_index()
just_referrers['state_rank'] = just_referrers.groupby(['year', 'state'])['count'].rank(ascending=False, method='min').astype('int64')

just_referrers = just_referrers.loc[just_referrers.state_rank==1]

just_referrers = just_referrers.groupby(['year','state'])['referrer'].apply(lambda x: ', '.join(x)).reset_index()
just_referrers['referrer'] = just_referrers['referrer'].apply(lambda x: 'Multiple' if ', ' in x else x)

just_referrers = just_referrers.pivot(index='state', columns='year', values='referrer')
just_referrers.columns.name = ''

just_referrers.to_excel('./flourish/ReferrerByState.xlsx')